In [ ]:
# Load python packages - pep 8 specifies core packages first... third party next
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import earthpy as et
import earthpy.plot as ep
from shapely.geometry import mapping

# Prettier plotting
import seaborn as sns
sns.set(font_scale=1.3)
sns.set_style("white")


et.data.get_data("colorado-flood")
os.chdir(os.path.join(et.io.HOME,
                      'earth-analytics',
                      'data'))

## Open Raster Data  Using Xarray

In [ ]:
# Because I may use the path to the data several times, i'm constructing the path in 2 steps!
data_path = os.path.join("colorado-flood",
                         "spatial",
                         "boulder-leehill-rd")
dtm_path = os.path.join(data_path,
                        "pre-flood",
                        "lidar",
                        "pre_DTM.tif")


lidar_dtm_pre = rxr.open_rasterio(dtm_path)
lidar_dtm_pre

## Plot The Data 

Notice that the data below look funky or don't import -- oops...

In [ ]:
f, ax = plt.subplots()
lidar_dtm_pre.plot(ax=ax)
plt.show()

# What Does a Histogram Show You?

In [ ]:
# What do you notice about the histogram of the data?
f, ax = plt.subplots()
lidar_dtm_pre.plot.hist(ax=ax)
plt.show()

In [ ]:
# This is elevation data- what do you notice
np.nanmin(lidar_dtm_pre), np.nanmax(lidar_dtm_pre)

In [ ]:
# View the no data value (sometimes referred to as a fill value)
lidar_dtm_pre.rio.nodata

In [ ]:
lidar_dtm_pre.shape

## Open the data and mask no data values

Below you open the data and mask no data values. YOu also use
squeeze to get rid of that extra first dimension which is the value
`1`. The array only has one layer so the additional dimension 
is not needed

In [ ]:
lidar_dtm_pre = rxr.open_rasterio(dtm_path,
                                  masked=True).squeeze()
print("The shape is", lidar_dtm_pre.shape)
print("The min and max values are: ",
      np.nanmin(lidar_dtm_pre),
      np.nanmax(lidar_dtm_pre))

In [ ]:
# Plot again
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 5))
lidar_dtm_pre.plot.imshow(ax=ax1)
lidar_dtm_pre.plot.hist(ax=ax2)

plt.show()

In [ ]:
# View the shape or the pixel dimensions of the raster
# Note that .shape is a numpy array ATTRIBUTE
lidar_dtm_pre.shape

## Create a Histogram of Raster Data
frequency  -- of elevations values

In [ ]:
# Remember that you don't want to use plt.show() in your homework!!
f, ax = plt.subplots()
lidar_dtm_pre.plot.hist(ax=ax,
                        bins=30,
                        color="purple")
ax.set(xlabel="Elevation (m)",
       ylabel="Count")
plt.show()

## Open Digital Surface Model from the Pre Flood Data

In [ ]:
# Open up pre-flood DSM
dsm_pre_path = os.path.join(data_path,
                            "pre-flood",
                            "lidar",
                            "pre_DSM.tif")

lidar_dsm_pre = rxr.open_rasterio(dsm_pre_path, masked=True).squeeze()

lidar_dsm_pre

In [ ]:
print("The shape is", lidar_dsm_pre.shape)
print("The min and max values are: ",
      np.nanmin(lidar_dsm_pre),
      np.nanmax(lidar_dsm_pre))

## Subtract the Rasters

In [ ]:
# subtract the dtm from the DSM to create a CHM
# IMPORTANT: be sure to review the lidar lessons!!
lidar_chm_pre = lidar_dsm_pre - lidar_dtm_pre
# view min and max value of the calculated CHM -- numpy array attributes
np.nanmin(lidar_chm_pre), np.nanmax(lidar_chm_pre)

In [ ]:
# the output calculated image is a xarray object
lidar_chm_pre

In [ ]:
# The commented line below can be used if you want to interact with your raster (mouse over to view values)
# %matplotlib notebook
# TODO: This is now throwing a runtime warning... Your instructor is looking into why!

# Plot the data - chm data
lidar_chm_pre.plot.imshow(
    cmap="Greens")
plt.show()

## Classify The Canopy Height Model

Classify in this instance refers to reassigning values in an existing raster to another value.

In [ ]:
# plot a histogram of the data but specify the bins you'd like to see
lidar_chm_pre.plot.hist(
    bins=[0, 2, 7, 12, 30],
    color="purple")
plt.show()  # remember to not use plt.show() in your homework as it messes up the autograder

In [ ]:
type(lidar_chm_pre)

In [ ]:
# define the bins to use in classification
class_bins = [-np.inf, 2, 7, 12, np.inf]
# classify your raster using numpy digitize
# Remember that digitize is simple reassigning existing values to a new value as specified
# in your class_bins list above!

lidar_chm_pre_class = xr.apply_ufunc(np.digitize,  # this is the function that you wish to call on the data
                                     lidar_chm_pre,  # your xarray object
                                     class_bins)  # the bins you wish to use to digitize
lidar_chm_pre_class.values

In [ ]:
# View unique values in the data
# Notice that now the values span from 1-5.
# Also note that the value 1 is likely no data as it's < 0 in the class bins above
np.unique(lidar_chm_pre_class)

In [ ]:
# Look at the data - what do you notice?
lidar_chm_pre_class.plot.imshow()

In [ ]:
# Mask cells with a value of 5 which is a fill value
pre_lidar_chm_class_ma = lidar_chm_pre_class.where(lidar_chm_pre_class != 5)
pre_lidar_chm_class_ma.plot.imshow()
plt.show()

## Add a Custom Legend to Your Plot with EarthPy

Note that the lessons online need to be updated!! I will do this tonight or tomorrow but you can use this code as an example of how legends work with earthpy. You can also look at the earthpy docs!!

In [ ]:
# This is how you can create your own custom colormap!
# This is optional but good to know how to do
colors = ['linen',
          'lightgreen',
          'darkgreen',
          'maroon']
height_class_labels = ["short trees",
                       "medium trees",
                       "tall trees",
                       "really tall trees"]

# Create a colormap from a list of colors
cmap = ListedColormap(colors)

f, ax = plt.subplots()
# Turn off the default colorbar and add an earthpy legend
im = pre_lidar_chm_class_ma.plot.imshow(add_colorbar=False)
ep.draw_legend(im, titles=height_class_labels)
# This line below turns off the ticks and black borders on your image
ax.set_axis_off()
plt.show()

In [ ]:
# Add labels

f, ax = plt.subplots()
# Turn off the default colorbar and add an earthpy legend
im = pre_lidar_chm_class_ma.plot.imshow(add_colorbar=False)
ep.draw_legend(im, titles=height_class_labels)
# This line below turns off the ticks and black borders on your image
ax.set(title="Title Here")
ax.set_axis_off()
plt.show()

## Crop a raster using rioxarray

The example above ran you through several raster operations. the one thing it skipped was cropping the data prior to processing it. You can crop when you import the data initially with rasterio as seen below. For your homework, you should do this when you import each raster dataset!

In [ ]:
# Open the crop extent that need to crop the data!
clip_extent_path = os.path.join(data_path,
                                "clip-extent.shp")
crop_extent = gpd.read_file(clip_extent_path)

In [ ]:
# View the cropped data layer
crop_extent.plot()
plt.show()

In [ ]:
# overlay the two datasets
# Note that the crop extent will remove some of the uneven edge on the left (West) side of the image
f, ax = plt.subplots()
pre_lidar_chm_class_ma.plot.imshow(
    ax=ax)
crop_extent.boundary.plot(ax=ax,
                          edgecolor="lightgreen",
                          color=None,
                          linewidth=3)
plt.show()

In [ ]:
# Crop the chm using rioxarray

pre_chm_clip = pre_lidar_chm_class_ma.rio.clip(crop_extent.geometry.apply(mapping),
                                                              # This is needed if your GDF is in a diff CRS than the raster data
                                                              crop_extent.crs)


# Plot the cropped data
f, ax=plt.subplots()
pre_chm_clip.plot.imshow(ax=ax)

# Use this is you want to turn of scientific notation for your plots
ax.ticklabel_format(useOffset=False, 
                    style='plain')
ax.set(title="Cropped CHM Data")
plt.show()